# Importing Required libraries

In [9]:
%run ./EODHD_Functions.ipynb
import requests
import pandas as pd
import datetime as dt
import numpy as np
import os as os
from textblob import TextBlob
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import matplotlib.pyplot as mdates
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import numpy as np

# Functions used

In [16]:
def calculate_30d_volatility(df, price_col='adjusted_close'):
    """
    Calculate the 30-day annualized volatility of a stock from a DataFrame of prices.
    Assumes daily frequency.
    """
    # Calculate log returns
    df = df.copy()
    df['log_return'] = np.log(df[price_col] / df[price_col].shift(1))
    # Use the last 30 log returns
    last_30 = df['log_return'].dropna()[-30:]
    # Calculate standard deviation and annualize (sqrt(252) for trading days)
    volatility = last_30.std() * np.sqrt(252)
    return volatility

# Creating blackscholes option pricing model

In [ ]:

def black_scholes(S, K, T, r, sigma):
    """
    Calculate Black-Scholes option price for European call and put.

    Parameters:
    S : float
        Current stock price
    K : float
        Strike price
    T : float
        Time to maturity in years
    r : float
        Risk-free interest rate (annual)
    sigma : float
        Volatility of the underlying asset (annual)

    Returns:
    call_price : float
        Price of the European call option
    put_price : float
        Price of the European put option
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return call_price, put_price

In [ ]:
# Using black-scholes model to calculate option price

Function = EODHD_Functions()
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=30)
s = Function.Fetch_historical_price("CA.PA", start_date,end_date,"d").iloc[-1]
T = 30/365
k = 12
r = 1.05
sigma = calculate_30d_volatility(Function.Fetch_historical_price("CA.PA", start_date,end_date,"d"))

CA_option_price = black_scholes(s['adjusted_close'], k, T, r/100, sigma)

print(f"Call Option Price: {CA_option_price[0]:.2f}, Put Option Price: {CA_option_price[1]:.2f}")

Call Option Price: 0.37, Put Option Price: 0.53
